# Embedding
![rag_embedding](figures/rag_embedding.png)

- 분할된 텍스트를 벡터 표현(임베딩 벡터)으로 변환한다.
- LangChain은 OpenAI, HuggingFace 등 다양한 임베딩 모델을 지원하며, 동일한 인터페이스로 사용할 수 있다.
- [임베딩모델의 메서드](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.embeddings.Embeddings.html#langchain_core.embeddings.embeddings.Embeddings)

    - **`embed_documents(texts: List[str])`**
        - 여러 문서를 받아 벡터화(임베딩)한다.
        - Context를 벡터화 할 때 사용한다.
    - **`embed_query(text: str)`**
        - 하나의 문자열(문서)을 받아 벡터화한다.
        - Query를 벡터화 할 때 사용한다.


In [1]:
from dotenv import load_dotenv

load_dotenv()   

True

In [ ]:
#pip install langchain-ollama
#pip show langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings

In [25]:
# # https://platform.openai.com/docs/models#embeddings
# embedding_model_name = "text-embedding-3-small"
# embedding_model = OpenAIEmbeddings(
#     model=embedding_model_name
# )


# ## HuggingFace의  Embeddings 모델 사용
# # huggingface.co : Models => task:NLP - Feature Extraction task를 선택
# embedding_model_name = "BAAI/bge-m3"
# embedding_model = HuggingFaceEmbeddings(
#     model_name = embedding_model_name
# )

## Ollama의  Embeddings 모델 사용
### ollama.com
embedding_model_name = "nomic-embed-text"
embedding_model = OllamaEmbeddings(
    model=embedding_model_name
)


In [26]:
# 문장(문서)들을 Embedding Vector로 변환
text_list = [
        "The meaning of COW is the mature female of cattle (genus Bos).",
        "A puppy is a juvenile dog.",
        "Which one is correct, 'many people' or 'much people'?",
        "Book your Amtrak train and bus tickets today by choosing from over 30 U.S.",
]
embedding_docs = embedding_model.embed_documents(text_list)
print(len(embedding_docs))


4


In [27]:
print(type(embedding_docs), type(embedding_docs[0]))
print("embedding vector의 차원수:", len(embedding_docs[0]))   

<class 'list'> <class 'list'>
embedding vector의 차원수: 768


In [31]:
embedding_docs[0][:10]

[-0.0099094855,
 0.042601053,
 -0.1549793,
 -0.05094128,
 0.07207404,
 0.038724188,
 -0.0276544,
 -0.026862217,
 -0.057049092,
 -0.03338403]

In [28]:
# 질문 -> Embedding Vector 
query = "How much the bus ticket price?"
embedding_query = embedding_model.embed_query(query) # 한 문장을 변환.
print(type(embedding_query), len(embedding_query))

<class 'list'> 768


In [29]:
# 코사인 유사도 계산 함수
import numpy as np
def cosine_similarity(v1:np.ndarray|list, v2:np.ndarray|list)->float:
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# np.linalg.norm(v1) 
# (v1) = [2, 3, 5]
# np.sqrt(2**2 + 3**2 + 5**2)

In [30]:
for i, e_vector in enumerate(embedding_docs):
    print(i, cosine_similarity(e_vector, embedding_query)) # 각 문서 - 질문 간의 유사도

0 0.29334711657233364
1 0.32038168592552274
2 0.3805105991454101
3 0.6073879825242146


# 벡터 데이터베이스(Vector Database)
- Embedding 된 문서를 Vector Database(Vector Store)에 저장한다.
- 이후 질문(Query)와 관련된 내용을 유사도를 이용해 검색해 질문과 함께 prompt로 만든다. (Retrieve)

![rag_vector_store](figures/rag_vector_store.png)

## 벡터 데이터베이스란
- 벡터 임베딩을 저장하고 관리하는 데이터베이스를 의미한다.
- 모든 데이터는 적절한 임베딩 모델을 활용하면 임베딩 벡터로 변환할 수 있다. 이렇게 변환된 임베딩 벡터를 벡터 데이터베이스에 저장하면 **임베딩 벡터 간의 거리 계산을 통해 데이터 간 유사도를 검색할 수 있다.**
    - 이미지, 텍스트, 음성 등 비정형 데이터를 임베딩 모델로 벡터화한 뒤 데이터베이스에 저장한다.
    - 벡터 간의 유사도 계산을 통해 연관성 있는 데이터나 유사한 데이터를 효과적으로 검색할 수 있다.
    - 좋은 검색 결과를 위해서는 벡터의 품질이 중요하다. 그래서 **임베딩 모델(Embedding Model)을 잘 선택하는 것이 중요**하다.
- 벡터 데이터베이스는 이러한 벡터 간 거리 계산에 특화된 데이터베이스다.

## 주요 특징

- 고차원 벡터 저장
  -  벡터 데이터베이스는 수백에서 수천 차원에 이르는 벡터 데이터를 효율적으로 저장하고 관리한다. 
  -  전통적인 데이터베이스로는 어려운 고차원 벡터 간 유사도 검색을 효율적으로 수행한다.
- 유사성 기반 검색
  -  벡터 간의 거리를 측정하여 유사한 데이터를 빠르게 검색할 수 있다. 
  -  일반적으로 사용되는 거리계산기법은 다음과 같다.
     - 코사인 유사도(Cosine Similarity)
     - 유클리드 거리(Euclidean Distance)
     - 맨하탄 거리(Manhattan Distance) 
- 비정형 데이터 처리: 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 변환하여 저장하고, 이러한 데이터를 효과적으로 검색할 수 있다.

## 벡터 데이터베이스와 딥러닝
- 벡터 데이터베이스는 딥러닝 기술의 발전과 깊은 관련이 있다.
- 딥러닝 모델은 학습 과정에서 데이터의 특징을 추출하는 방법을 함께 학습한다. 충분한 데이터를 학습한 딥러닝 모델은 데이터의 특성을 추론하기 위한 **특성 벡터(feature vector)**를 효과적으로 생성할 수 있다.
- 이때 추출된 특성 벡터는 고차원 데이터를 저차원 공간에서 표현한 **임베딩 벡터**다.
    - > **임베딩**은 고차원 데이터를 저차원 공간으로 변환하여 표현하는 방법으로, 정보 손실을 최소화하면서 데이터 간의 의미 있는 관계를 벡터 공간에서 유지한다.
- 딥러닝 모델로 추출한 데이터의 특징을 임베딩 공간에 배치하면, 비슷한 데이터는 가까이, 그렇지 않은 데이터는 멀리 배치된다.
- 이러한 특성을 활용하면 임베딩 벡터 간의 거리를 계산해 유사한 데이터를 효과적으로 검색할 수 있다. 벡터 데이터베이스는 이러한 임베딩 벡터의 특성을 기반으로 개발되었다.
- 딥러닝 기술의 발전과 폭넓은 활용으로 임베딩 데이터의 사용이 증가하면서, 이를 저장하고 관리하는 기능에 특화된 데이터베이스에 대한 수요도 증가해 다양한 벡터 데이터베이스가 등장했다.

## 벡터 데이터베이스의 주요 기능
1. **저장**  
   - 이미지, 텍스트, 음성 등 **비정형 데이터**를 임베딩 모델을 통해 벡터로 변환한 뒤 벡터 데이터베이스에 저장한다.
2. **검색**  
   - 검색하려는 데이터를 임베딩 모델로 변환한 뒤, 벡터 데이터베이스에서 유사도를 기반으로 검색한다.
3. **결과 반환**  
   - 벡터 데이터베이스는 저장된 벡터 중 검색 쿼리 임베딩과 가장 가까운 벡터를 찾아 반환한다.

## LLM과 벡터 데이터베이스
- ChatGPT의 등장 이후 벡터 데이터베이스는 폭발적인 주목을 받았다.
- 임베딩 벡터의 유사도를 기반으로 문서를 검색하는 RAG(Relevant Augmented Generation) 기술은 LLM의 환각(할루시네이션) 현상을 줄이고, LLM을 추가 학습하지 않고도 최신 정보를 효율적으로 활용할 수 있는 핵심 기법으로 자리 잡았다.
   


## 벡터 데이터베이스 종류
![img](figures/vector_database.png)

<<https://blog.det.life/why-you-shouldnt-invest-in-vector-databases-c0cd3f59d23c>>

### 주요 벡터 데이터베이스 종류
- **Pinecone**
    - 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 간단한 API를 통해 벡터 데이터를 관리할 수 있다.  
    - 자동 확장성과 고가용성을 제공하며, 실시간 데이터 수집과 유사성 검색에 최적화되어 있다.
    - 가장 쉽게 시작할 수 있는 관리형 서비스를 제공한다.
- **Chroma**
    - 벡터 임베딩을 효율적으로 저장하고 검색할 수 있는 오픈소스 데이터베이스로, AI 및 머신러닝 애플리케이션에 최적화되어 있다.
    - 대규모 임베딩 저장에 최적화되어 있다.
- **FAISS**
    - Facebook AI에서 개발한 고성능 벡터 검색 라이브러리로, 고차원 벡터의 효율적인 유사성 검색을 위해 최적화되어 있다.
    - GPU를 활용해 계산 성능을 높이며, 벡터 양자화 기술을 활용하여 메모리 사용을 최적화한다.
    - 근사 최근접 이웃 검색(ANNS)에 최적화되어 있다.
- **Milvus**
    - 오픈소스 벡터 데이터베이스로, 대규모 벡터 데이터를 효율적으로 저장하고 검색할 수 있다.  
    - 분산 아키텍처를 채택하여 확장성이 뛰어나며, IVF_PQ, DiskANN 등 다양한 인덱싱 알고리즘을 지원한다.
    - 대규모 데이터셋 처리에 가장 적합한 솔루션이다.
- **Weaviate**
    - 오픈소스 벡터 데이터베이스로, 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 저장하고 검색할 수 있다.  
    - GraphQL API를 통해 접근 가능하며, 내장된 머신러닝 모듈을 통해 가장 강력한 의미론적 검색 기능을 제공한다.
- **Qdrant**
    - Rust로 개발된 고성능 벡터 검색 엔진으로, 실시간 근사 최근접 이웃 검색을 제공한다.  
    - 추천 시스템에 특화되어 있으며, 벡터 임베딩 저장과 유사도 쿼리를 효율적으로 수행한다.
- **Elasticsearch**
    - HNSW 알고리즘을 사용하여 벡터 검색을 구현하는 검색 엔진이다.
    - 전통적인 검색 기능과 벡터 검색을 효과적으로 결합할 수 있어, 하이브리드 검색에 가장 적합하다.
- **PGVector**
    - PostgreSQL의 확장 모듈로, 벡터 데이터를 저장하고 유사성 검색을 수행할 수 있게 해준다.  
    - SQL과 통합된 벡터 연산이 가능하며, L2 거리, 코사인 거리, 내적 등 다양한 거리 측정 방식을 지원한다.


# Langchain - Vector Store 연동 
- Langchain은 다양한 벡터 데이터베이스와 연동할 수 있다.
- 벡터 데이터베이스 마다 API가 다르기 때문에, Langchain을 사용하면 동일한 interface로 사용할 수 있다.

## **VectorStore**
- Langchain이 지원하는 모든 벡터 데이터베이스는 **VectorStore** 인터페이스를 구현한다.
- 그래서 Langchain에서는 벡터 데이터베이스를 **Vector Store** 라고 한다.
- https://python.langchain.com/docs/integrations/vectorstores/

## InMemoryVectorStore
- langchain-core에서 제공하는 메모리 기반 벡터 데이터베이스이다.
- Data들을 Dictionary를 사용해 메모리에 저장하며, 검색 할 때 코사인 유사도(cosine similarity)를 계산하여 조회한다.
- 설치
  - `pip install -qU langchain-core`

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from pprint import pprint
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# OpenAI에서 제공하는 임베딩 모델을 사용하기 위한 설정.
# "text-embedding-3-small" 모델은 상대적으로 가볍고 빠른 임베딩 생성을 제공.
# Embedding 모델은 성능이 좋은 것을 사용.
embedding_model=OpenAIEmbeddings(model= "text-embedding-3-small")
vector_store = InMemoryVectorStore(embedding_model)
# InMemoryVectorStore는 메모리 기반의 벡터 저장소로, 
# 임베딩 데이터를 메모리에 저장하여 빠른 검색이 가능하게 한다.
# 로컬 환경에서 소규모 데이터셋을 처리하기에 적합.

In [ ]:
from langchain_core.documents import Document

# 각 문서를 Document 객체로 생성.
# id: 문서 고유 식별자
# page_content: 문서의 주요 내용
# metadata: 문서와 관련된 부가 정보
doc_1 = Document(id="1", page_content="Apple, Pear, Watermelon", metadata={"category":"Fruit"})

doc_2 = Document(id="2", page_content="Pyton, C++, Javam C#", metadata={"category":"IT"})

doc_3 = Document(id="3", page_content="Football, Baseball, Basketball", metadata={"category":"Sports"})

# Document 객체들을 하나의 리스트로 묶어서 관리.
# Document 들을 List로 묶어준다.
docs = [doc_1, doc_2, doc_3]

# Vector Store에 문서를 추가.
# 문서의 page_content를 임베딩(embedding)하여 저장하고,
# metadata와 id는 그대로 저장되어 나중에 검색 및 조회에 활용.
# Vector Store에 문서들을 저장 -> page_content를 embedding해서 저장. metadata, id는 그대로 저장.
vector_store.add_documents(documents=docs)


# 위 코드 흐름:
# 1. Vector Store 생성: 메모리 기반으로 임베딩 벡터를 저장하는 저장소 초기화.
# 2. 생성된 Vector Store에 데이터를 저장(추가): 문서 리스트를 추가하여 검색에 필요한 임베딩 벡터 구성.

# 1. Vector Store 생성.   2. 생성된 Vector Store 에 데이터를 저장(추가)


['1', '2', '3']

In [5]:
# 2. Vector Store를 데이터에 추가하면서 생성.
vector_store2 = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [ ]:
# 검색 -> 유사도 검색
query = "SQL"
query = "Rust"
query = "Orange"
# query = "Volleyball"
# query = "House"
# result = vector_store.similarity_search(
result = vector_store.similarity_search_with_score(  # 유사도 점수를 포함해서 리턴.
    query=query,   # 검색 대상 문자열
    k=2,           # 유사도가 높은 순서대로 지정한 갯수의 데이터를 반환함.
)

In [7]:
result

[(Document(id='1', metadata={'category': 'Fruit'}, page_content='Apple, Pear, Watermelon'),
  0.1622070321836628),
 (Document(id='3', metadata={'category': 'Sports'}, page_content='Football, Baseball, Basketball'),
  0.15183523432231988)]

# 실습
- data/olympic.txt 
1. loading
2. split
3. embedding + vector store(InMemoryVectorStore)에 저장
4. query(질의)

In [8]:
from langchain_community.document_loaders import TextLoader
# TextLoader 로 document loading

path ="data/olympic.txt"
loader = TextLoader(path, encoding="utf-8")
load_docs = loader.load()
print(len(docs))

3


In [9]:
# RecursiveTextCharacterTextSplitter.from_titoken_encoder()

from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",  
    chunk_size=500,   
    chunk_overlap=100,
)
docs = splitter.split_documents(load_docs)


In [10]:
#### load와 split 을 한번에 처리

path ="data/olympic.txt"
loader = TextLoader(path, encoding="utf-8")
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",  
    chunk_size=500,   
    chunk_overlap=100,
)
docs = loader.load_and_split(splitter)
print(len(docs))

# docs list[document]

36


In [11]:
# InMemoryVectorStore 생성 및 저장
## 생성과 저장을 한번에 처리  

from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = InMemoryVectorStore.from_documents(
    embedding=embedding_model,
    documents=docs
)

In [12]:
# similarity_search()를 이용해서 질문과 관련된 문서들을 조회.

# query = "동계 올림픽에 대해 설명해주세요."
query = "올림픽에 금메달을 가장 많이 획득한 선수를 알려주세요."
result = vector_store.similarity_search_with_score(
    query=query,  
    k=3,          
)
# result : list[tuple[Document, 점수]]

In [13]:
for document, score in result:
    print("유사도점수:", score)
    print(document.page_content[:1000])
    print("=======================================================")

유사도점수: 0.48681135541523596
올림픽은 거의 모든 국가가 참여할 정도로 규모가 커졌다. 하계 올림픽은 33개의 종목과 약 400개의 세부종목에서 13,000명이 넘는 선수들이 겨루고 그중 각 종목별 1, 2, 3위는 각각 금/은/동을 수여받는다. 전 세계 언론에서 각각 4년마다 열리는 올림픽 경기를 중계하기 때문에 이름 없는 선수가 개인적, 국가적, 세계적으로 명성을 얻을 수 있는 기회가 된다. 이와 더불어 올림픽 경기는 개최지와 개최국에게도 전 세계에 그 이름을 널리 알리는 좋은 기회가 된다.
유사도점수: 0.4528750282777687
올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가

## MMR(최대 한계 관련성-Maximal Marginal Relevance) 알고리즘 적용
최대 한계 관련성(Maximal Marginal Relevance, MMR) 알고리즘은 정보 검색 및 요약에서 검색 결과의 **관련성**과 **다양성**을 동시에 고려하여 최적의 결과를 제공하는 방법이다. 
이 알고리즘은 사용자 쿼리와의 관련성을 최대화하면서도 중복 정보를 최소화하여 다양한 정보를 제공하는 것을 목표로 한다.

1. **관련성과 다양성의 균형 조절**: MMR은 사용자 쿼리와 문서 간의 유사성 점수와 이미 선택된 문서들과의 다양성 점수를 조합하여 각 문서의 최종 점수를 계산한다. 이를 통해 관련성이 높으면서도 중복되지 않는 문서를 선택한다.

2. **수학적 정의**: MMR은 다음과 같이 정의됩니다:
   $$
   \text{MMR} = \lambda \cdot \text{Sim}(d, Q) - (1 - \lambda) \cdot \max_{d' \in D'} \text{Sim}(d, d')
   $$

   - $\text{Sim}(d, Q)$: 문서 $d$와 쿼리 $\text{Q}$ 사이의 유사성. (문서 유사성 계산)
   - $\max_{d' \in D'} \text{Sim}(d, d')$: 문서 $d$와 이미 선택된 문서 집합 $D'$ 중 가장 유사한 문서와의 유사성. (문서 다양성 계산)
   - $\lambda$: 유사성과 다양성의 중요도를 조절하는 매개변수(parameter)
3. **적용 분야**: MMR은 정보 검색, 추천 시스템, 문서 요약 등에서 활용된다. 특히 LLM 검색에서 성능 향상이 입증되었다.

### `vectorStore.max_marginal_relevance_search()` 메소드
  - MMR 알고리즘을 적용한 검색을 수행한다.
  - **파라미터**
    - **query**: 사용자로부터 입력받은 검색 쿼리
    - **k**: 최종적으로 선택할 문서의 수
    - **fetch\_k**: MMR 알고리즘 적용 시 고려할 상위 문서의 수
    - **lambda_mult**: 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절하는 매개변수. $\lambda = 1$이면 유사성만 고려하고, $\lambda = 0$이면 다양성만을 최대화한다.
    - **filter**: 검색 결과를 필터링할 조건을 지정한다.


In [55]:
from dotenv import load_dotenv
load_dotenv()

True

In [56]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 문서 Load data 및 split
loader = TextLoader("data/olympic.txt", encoding="utf-8")
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
# splitter = RecursiveCharacterTextSplitter(
    model_name="gpt-4o-mini",
    chunk_size=200
)
docs = loader.load_and_split(splitter)
len(docs)


876

In [57]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 문서 load 및 split
loader = TextLoader("data/olympic.txt", encoding="utf-8")
# splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(   alt+shift + 위/아래화살표 (라인복사)
splitter = RecursiveCharacterTextSplitter(
    # model_name="gpt-4o-mini", 
    chunk_size=500
)
docs = loader.load_and_split(splitter)
len(docs)

63

In [58]:
##### GPT 서버 다운으로 사용 못함.. 2024.12.12

COLLECTION_NAME = "olympic_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/olympic_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

# EMBEDDING_MODEL_NAME = "nomic-embed-text"
# embedding_model = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)

# Vector DB 생성
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [59]:
vector_store._collection.count()

99

In [60]:
query = "국제 올림픽 위원회에 대해 설명해주세요."
result = vector_store.similarity_search_with_score(
    query=query,
    k=4
)
for doc, score in result:
    print(score, doc)

0.7040834422930683 page_content='국제 올림픽 위원회
올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.
- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)
- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.
- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.
올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.' metadata={'source': 'data/olympic.txt'}
0.7486046687834373 page_content='국제 올림픽 위원회
올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한

- 유사도가 높은 것만 확인함으로써, 다양성이 떨어지는 데이터만 가져오게 됨.

In [61]:
## MMR를 이용한 조회
result = vector_store.max_marginal_relevance_search(
    query=query,
    k=4,
    fetch_k=10,         # 윺사도 높은 것으로 10개를 찾은 뒤 , 각각의 유사도를 확인하여 다양성을 적용시킴.
    lambda_mult=0.5,    # 유사도와 다양성을 어떤 비율로 적용할지의 값. 1: 유사도, 0: 다양성  (default= 0.5,  0<----->1)
)


In [62]:
for doc in result:
    print(doc)

page_content='국제 올림픽 위원회
올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.
- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)
- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.
- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.
올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.' metadata={'source': 'data/olympic.txt'}
page_content='올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올

In [65]:
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

system_message = """
당신은 question-answering 업무를 수행하는 유능한 인공지능 비서입니다.
제공되는 문서를 바탕으로 질문에 대답해주세요.
답을 모르는 질문에는 모른다고 솔직하게 답변해주세요.
"""

prompt_template = ChatPromptTemplate(
    [
        ("system", system_message),
        ("human", "[질문관련 문서]\n{context}\n\n[질문]\n{question}")
    ]
)
MODEL_NAME = "gpt-4o-mini"
model = ChatOpenAI(
    model=MODEL_NAME
)


# MODEL_NAME = "qwen2:0.5b"
# model = ChatOllama(
#     model=MODEL_NAME
# )

chain = prompt_template | model | StrOutputParser()

In [67]:
result

[Document(metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)\n- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.\n올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다.

In [68]:
# vector_store 에서 query와 연관련 문서들을 조회 -> prompt template 에 넣는 형식으로 변환.
print(query)
print(result)

국제 올림픽 위원회에 대해 설명해주세요.
[Document(metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)\n- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.\n올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.'), Document(metadata={'source': 'data/olympic.txt'}, page_content='올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예

In [69]:
context = ""
for res in result:
    context += res.page_content+"\n\n"
print(context)

국제 올림픽 위원회
올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.
- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)
- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.
- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.
올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.

올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올림픽 경기 개최 도시는 경기 축하 의식이 올림픽 헌장에 부합하도록 조직하고 기금을 마련해야 한다. 올림픽 축하 행사로는 여

In [70]:
final_result = chain.invoke({"context":context, "question":query})

In [71]:
print(final_result)

국제 올림픽 위원회(IOC)는 올림픽 활동을 통솔하는 단체로, 다양한 국가와 국제 경기 연맹, 협회와 미디어 파트너를 맺고 있습니다. IOC는 올림픽 개최 도시를 선정하고, 계획을 감독하며, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권한을 가지고 있습니다. 

IOC는 국제경기연맹(International Federation, IF), 국가 올림픽 위원회(National Olympic Committee, NOC), 그리고 올림픽 조직 위원회(Organizing Committee for the Olympic Games, OCOG)로 구성된 올림픽 생태계를 관리합니다. IOC는 올림픽 종목을 결정하고, 올림픽 개최 도시에서의 경기 축하 의식을 조직하도록 요구합니다. 

올림픽의 공식 언어는 프랑스어와 영어, 그리고 개최국의 공용어입니다. IOC는 정치적 주권국에 관계없이 모든 공동체가 참가할 수 있도록 허용하며, 현재 IOC에 소속된 국가 올림픽 위원회는 205개입니다. IOC는 또한 '올림픽 프로그램 위원회'를 통해 올림픽 종목과 비올림픽 스포츠를 재검토하며, 특정 기준을 충족하는 종목을 올림픽에 포함시키는 작업을 합니다.
